In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('Cleaned-P&P data.csv')

blanks = []  # start with an empty list`

for i,c,d,inc,sd in df1.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
df1.drop(blanks, inplace=True)
df1["desc"] = df1["short_des"] + '. ' + df1["desc"]
df1.reset_index(drop=True,inplace=True)


In [3]:

df2 = pd.read_csv('Cleaned-P&P data.csv')
blanks2 = []  # start with an empty list`

for i,c,d,inc,sd in df2.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks2.append(i)     # add matching index numbers to the list
df2.drop(blanks2, inplace=True)
df2.reset_index(drop=True,inplace=True)

In [4]:
dataset = pd.concat([df1, df2], ignore_index=True)

In [5]:
# Cleaning the texts
import re
corpus = []
all_words = []
max_len=0
for i in range(0, 488):

    review = re.sub('\w\d{7}', ' ', dataset['desc'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    
    review = nlp(review)
    review = [word.text for word in review]
   # for word.text in review:
        
    all_words.append(review) 
    if len(review) > max_len:
        max_len = len(review)
    ds = ' '.join(review)
#    for word2vec we want an array of vectors
    corpus.append(ds)
    

### Standardise key terms in data

In [6]:
dpf = pd.read_csv('Del-Pass.csv')



In [7]:
dpf['kpattern']

0     delivery subscription service
1        delivery pass subscription
2             delivery pass service
3                free delivery pass
4                   delivery passes
5                    delivery pass 
6        p&p delivery subscription 
7            delivery subscriptions
8             delivery subscription
9                      delivery sub
10                      p&p service
11                              p&p
12                     subscription
Name: kpattern, dtype: object

In [8]:
keys_list = dpf['kpattern'].tolist()
regex = re.compile(r'(' + '|'.join(keys_list) + r')') 

In [9]:
cus_list = ['remove','removed','cancelled', 'cancel' ,'deleted']
regey = re.compile(r'(' + '|'.join(cus_list) + r')') 

In [96]:
from nltk.corpus import stopwords
stopwords=nltk.corpus.stopwords.words('english')
additional =["thank you","please","hello","hi","advise","hin't","st","nd","rd","th","thank"]
stopwords = stopwords + additional
remov=[]
for ele in stopwords:
    matches = re.findall("n't",ele)
    matches2 = re.findall("'nt",ele)
    if len(matches)>0 or len(matches2)>0:
        remov.append(ele)
stopwords = [word for word in stopwords if word not in remov]

In [10]:
cldict = {'P':1 , 'R':0}
dataset['Class Label'] = dataset['class'].map(cldict)
label_series = pd.Series(dataset['Class Label'])

In [11]:
from keras.utils import to_categorical

class_label = np.array(label_series)
y = class_label
y.shape

Using TensorFlow backend.


(488,)

In [12]:
X = corpus

In [13]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47)


### Create custom rules 

In [100]:
#segment_featurizer = SegmentFeaturizer()  # more on this below
class CustomLinguisticFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        ref_corpus=[]
        for text in X:           
            trans = regex.sub(lambda m: m.group().replace(m.group(),"del-pass"),text)
            trans_ref = regey.sub(lambda n: n.group().replace(n.group(),"delete"),trans)
             
            ref_corpus.append(trans_ref)
       
        return ref_corpus

### Spacy Vector Transformer

In [101]:

from sklearn.base import BaseEstimator, TransformerMixin
class SpacyVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300
        pass
    def fit(self, X, y):
        return self

    def transform(self, X):
        # Doc.vector defaults to an average of the token vectors.
        # https://spacy.io/api/doc#vector
        lemmatized_list=[]
        for snt in X:
        #    print(snt)
            tokens = nlp(snt)
        #    for token in tokenized:
            filtered_sentence = [w.text for w in tokens if not w.text in stopwords]
        #                 lemm = token.lemma_ for token.text in token
        #             lemmatized_list.append(lemm)
            stri = ' '.join(filtered_sentence)
            lemmatized_list.append(stri)
        return [self.nlp(text).vector for text in lemmatized_list]
       


### Reduce dimensionality

In [18]:
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

embeddings_pipeline = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", RandomForestClassifier()),
    ]
)
embeddings_pipeline.fit(X_train, y_train)

C:\Users\AVNEET\miniconda3\envs\nlp_course\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('mean_embeddings', SpacyVectorTransformer(nlp=<spacy.lang.en.English object at 0x0000029F648E8B08>)), ('reduce_dim', TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, c...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

embeddings_pipeline2 = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", LinearSVC()),
    ]
)
embeddings_pipeline2.fit(X_train, y_train)

NameError: name 'Pipeline' is not defined

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
svm = LinearSVC()
embeddings_pipeline3 = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", CalibratedClassifierCV(svm)),
    ]
)
embeddings_pipeline3.fit(X_train, y_train)

In [102]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import TfidfVectorizer
svm = LinearSVC()
embeddings_pipeline4 = Pipeline(
    steps=[
        ("cust",CustomLinguisticFeatureTransformer()),
#        ("tfidf", TfidfVectorizer()),
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(200)),
#         ("dict_vect", DictVectorizer()),
        ("classifier", CalibratedClassifierCV(svm)),
    ]
)
embeddings_pipeline4.fit(X_train, y_train)

C:\Users\AVNEET\miniconda3\envs\nlp_course\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Pipeline(memory=None,
     steps=[('cust', CustomLinguisticFeatureTransformer()), ('mean_embeddings', SpacyVectorTransformer(nlp=<spacy.lang.en.English object at 0x0000029F648E8B08>)), ('reduce_dim', TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
       random_state=None, tol=0.0)), ('classifier', Calibrated...lty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv='warn', method='sigmoid'))])

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
#svm = LinearSVC()
embeddings_pipeline5 = Pipeline(
    steps=[
        ("cust",CustomLinguisticFeatureTransformer()),
       ("tfidf", TfidfVectorizer()),
#        ("mean_embeddings", SpacyVectorTransformer(nlp)),
#        ("reduce_dim", TruncatedSVD(50)),
#         ("dict_vect", DictVectorizer()),
#        ("rdf", RandomForestClassifier()),
        ("mnb",MultinomialNB()),
    ]
)
embeddings_pipeline5.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cust', CustomLinguisticFeatureTransformer()), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1)...rue,
        vocabulary=None)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [117]:

y_pred = embeddings_pipeline4.predict(X_test)
cr = classification_report(y_test, y_pred)

In [118]:
print(cr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00        65

   micro avg       1.00      1.00      1.00        98
   macro avg       1.00      1.00      1.00        98
weighted avg       1.00      1.00      1.00        98



In [119]:
list1 = [' ']
print(embeddings_pipeline4.predict(list1))
probs = embeddings_pipeline4.predict_proba(list1)
np.around(probs, decimals = 3)*100

[1]


array([[22.2, 77.8]])

In [121]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[33,  0],
       [ 0, 65]], dtype=int64)

In [123]:
accuracy_score(y_test, y_pred)

1.0

In [31]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, y_pred)

0.014344262295081968

In [32]:
from sklearn.model_selection import cross_val_score
cross_val_score(embeddings_pipeline5, X, y, cv = 10)

array([1.        , 1.        , 0.97959184, 1.        , 0.93877551,
       1.        , 1.        , 0.97959184, 1.        , 0.91666667])

In [ ]:
cv_mean_score = np.mean(cross_val_score(embeddings_pipeline3, X, y, cv = 10))
clf_score = embeddings_pipeline3.score(X_test, y_test)
cv_mean_score, clf_score

In [33]:
predictions = embeddings_pipeline5.predict(X)
len(X)

488

In [180]:
clf_score = embeddings_pipeline3.score(X, y)
clf_score

0.9959016393442623

In [34]:
Faulty_Xtest =[]
for i in range(0,488):
    if y[i] != predictions[i]:
        print(f'predicted class:{X[i]}', f'Index is : {i}')
        Faulty_Xtest.append(X[i])
arr = np.array(Faulty_Xtest)
arr

predicted class:pnt work requests hello service line please can you look into the cancel the following account the delivery sub is not working customerno itemno rv fg delivery sub purchase date please if you could reply back and keep us up to date regards ahmed hussain proactive notification team citrix i d managers name claire wrigley Index is : 171
predicted class:delivery pass needs removing on staff accounts supra dba req please can the delivery pass be removed from the attached staff accounts thank you Index is : 201
predicted class:hello service line please can you look into the cancel the following account the delivery sub is not working customerno itemno rv fg delivery sub purchase date please if you could reply back and keep us up to date regards ahmed hussain proactive notification team citrix i d managers name claire wrigley Index is : 415
predicted class:please can you activate the delivery pass on customers account thank you Index is : 435
predicted class:please can the de

array(['pnt work requests hello service line please can you look into the cancel the following account the delivery sub is not working customerno itemno rv fg delivery sub purchase date please if you could reply back and keep us up to date regards ahmed hussain proactive notification team citrix i d managers name claire wrigley',
       'delivery pass needs removing on staff accounts supra dba req please can the delivery pass be removed from the attached staff accounts thank you',
       'hello service line please can you look into the cancel the following account the delivery sub is not working customerno itemno rv fg delivery sub purchase date please if you could reply back and keep us up to date regards ahmed hussain proactive notification team citrix i d managers name claire wrigley',
       'please can you activate the delivery pass on customers account thank you',
       'please can the delivery pass be removed from the attached staff accounts thank you',
       'please can you r

In [102]:
dataset['inc_num'][201], dataset['desc'][201], dataset['class'][201]

('INC0612636',
 'delivery pass needs removing on staff accounts supra dba req. please can the delivery pass be removed from the attached staff accounts?thank you!',
 'P')

In [11]:
# Enter your code here:

for token in nlp(X_test[94]):
    print(f'{token.text:{12}} {token.pos_:{6}} {token.dep_:{6}} {spacy.explain(token.dep_)}')

hi           INTJ   intj   interjection
the          DET    det    determiner
customer     NOUN   nsubj  nominal subject
took         VERB   ccomp  clausal complement
the          DET    det    determiner
delivery     NOUN   compound None
subscription NOUN   dobj   direct object
out          PART   prt    particle
on           ADP    prep   prepositional modifier
th           DET    nummod None
november     NOUN   pobj   object of preposition
but          CCONJ  cc     coordinating conjunction
is           VERB   aux    auxiliary
still        ADV    advmod adverbial modifier
being        VERB   auxpass auxiliary (passive)
charged      VERB   conj   conjunct
delivey      NOUN   advmod adverbial modifier
please       INTJ   intj   interjection
advise       VERB   ROOT   None
thanks       NOUN   dobj   direct object
peter        X      dobj   direct object


In [216]:

from spacy import displacy
displacy.render(nlp(X_test[95]), style='ent', jupyter=True)

In [26]:
import re
for match in re.finditer("delivery subscription ",X_test[94]):
    X_test[94]
    print(match.span())


(25, 47)


In [95]:
lemmatized_list=[]
for snt in X[:5]:
    print(snt)
    tokens = nlp(snt)
#    for token in tokenized:
    filtered_sentence = [w.text for w in tokens if not w.text in stopwords]
#                 lemm = token.lemma_ for token.text in token
#             lemmatized_list.append(lemm)
    stri = ' '.join(filtered_sentence)
    lemmatized_list.append(stri)
lemmatized_list

good afternoon this customer is being charged for delivery after purchasing the subscription please can you see if it needs activating thank you in advance rebecca connolly proactive notification team
delivery pass needs removing on staff accounts orders supra dba req please can you remove the delivery pass be removed from the attached staff accounts thanks
delivery pass needs removing on staff accounts orders supra dba req please can the delivery pass be removed from the attached staff accounts thanks
customer being charged for delivery customer website jd williams mainframe issue she ordered on th april and th mayand she is being charged for delivery even after she has the subscription order number contact number
website web shop issues short description tamara called in to report that the customer purchase the delivery subscription on the st of marchbut she is still charged for delivery every time instead of having a delivery subscription affected people


['good afternoon customer charged delivery purchasing subscription see needs activating thank advance rebecca connolly proactive notification team',
 'delivery pass needs removing staff accounts orders supra dba req remove delivery pass removed attached staff accounts thanks',
 'delivery pass needs removing staff accounts orders supra dba req delivery pass removed attached staff accounts thanks',
 'customer charged delivery customer website jd williams mainframe issue ordered th april th mayand charged delivery even subscription order number contact number',
 'website web shop issues short description tamara called report customer purchase delivery subscription st marchbut still charged delivery every time instead delivery subscription affected people']

In [31]:
matche = re.findall("delivery subscription ",X_test[94])
match.span()

(25, 47)

In [27]:
regex = re.compile('([a-zA-Z]\"[a-zA-Z])', re.S)
myfile =  'foo"s bar'
myfile2 = regex.sub(lambda m: m.group().replace('"',"%",1), myfile)
print(myfile2)

foo%s bar


In [165]:


lst = ['delivery subscription service', 'delivery pass subscription','delivery pass service','delivery passes','delivery pass ','p&p delivery subscription ','delivery subscriptions','delivery subscription']

def replace_double_quote(match):
    text = match.group()
    print(match.group())
    return text.replace(text, 'del-pass')
regex = re.compile(r'(' + '|'.join(lst) + r')')
myfile = 'please can you apply the annual delivery subscription is activated on this customers account purchased still being charged for delivery thank you'
regex.sub(replace_double_quote, myfile)



delivery subscription


'please can you apply the annual del-pass is activated on this customers account purchased still being charged for delivery thank you'

In [164]:


lst = ['foo', 'bar', 'and']
def my_func():
    
    regex = re.compile(r'(' + '|'.join(lst) + r')')
    myfile = 'foo"s bar and bar"s foo'
   
    pxk = regex.sub(lambda m: m.group().replace(m.group(),"rap",1), myfile)
    return pxk
#     def replace_double_quote(match):
#         text = match.group()
#         print(text)
#         return text.replace(text, 'rap')
   
print(my_func())

rap"s rap rap rap"s rap


In [158]:

def match_kpatterns(match):
    
    text = match.group()
    print(match.group())
    return text.replace(text, 'del-pass')


regez = re.compile(r'(' + '|'.join(keys_list) + r')')
seg = "please can you apply the annual delivery subscription is activated on this customers account purchased still being charged for delivery thank you"

regez.sub(match_kpatterns, seg)
#    return regex.sub(lambda m: m.group().replace(m.group(),"del-pass"), rfseg)
#r_featurize(seg)

delivery sub


'please can you apply the annual del-passscription is activated on this customers account purchased still being charged for delivery thank you'

In [42]:
text = nlp(u'trashbin')
text1 = nlp(u'dustbin')
fr = text1.similarity(text)
fr

0.536630904249573

In [94]:
for i in range(0,488):
    matches = re.findall("n t ",corpus[i])
    print(f'mathc is : {matches}', i)

mathc is : [] 0
mathc is : [] 1
mathc is : [] 2
mathc is : [] 3
mathc is : [] 4
mathc is : [] 5
mathc is : [] 6
mathc is : [] 7
mathc is : [] 8
mathc is : [] 9
mathc is : [] 10
mathc is : [] 11
mathc is : [] 12
mathc is : [] 13
mathc is : [] 14
mathc is : [] 15
mathc is : [] 16
mathc is : [] 17
mathc is : [] 18
mathc is : [] 19
mathc is : [] 20
mathc is : [] 21
mathc is : [] 22
mathc is : [] 23
mathc is : [] 24
mathc is : [] 25
mathc is : ['n t ', 'n t '] 26
mathc is : [] 27
mathc is : [] 28
mathc is : [] 29
mathc is : [] 30
mathc is : [] 31
mathc is : [] 32
mathc is : [] 33
mathc is : [] 34
mathc is : [] 35
mathc is : [] 36
mathc is : [] 37
mathc is : [] 38
mathc is : [] 39
mathc is : [] 40
mathc is : [] 41
mathc is : [] 42
mathc is : [] 43
mathc is : [] 44
mathc is : [] 45
mathc is : [] 46
mathc is : ['n t '] 47
mathc is : [] 48
mathc is : [] 49
mathc is : [] 50
mathc is : [] 51
mathc is : [] 52
mathc is : [] 53
mathc is : [] 54
mathc is : ['n t '] 55
mathc is : [] 56
mathc is : [] 5

In [44]:
lemm=[]
#for snt in corpus[0]:
kr = nlp(corpus[0])
for token in kr:
    ls = token.lemma_
    lemm.append(ls)
lemm

['good',
 'afternoon',
 'this',
 'customer',
 'be',
 'be',
 'charge',
 'for',
 'delivery',
 'after',
 'purchase',
 'the',
 'subscription',
 'please',
 'can',
 '-PRON-',
 'see',
 'if',
 '-PRON-',
 'need',
 'activate',
 'thank',
 '-PRON-',
 'in',
 'advance',
 'rebecca',
 'connolly',
 'proactive',
 'notification',
 'team']

In [49]:
nlp(lemm[0]).vector

array([-0.42625  ,  0.4431   , -0.34517  , -0.1326   , -0.05816  ,
        0.052598 ,  0.21575  , -0.36721  , -0.04519  ,  2.2444   ,
       -0.29089  ,  0.1667   , -0.052051 ,  0.15964  , -0.42759  ,
       -0.11147  , -0.14951  ,  1.18     , -0.19603  ,  0.15592  ,
       -0.06112  , -0.011576 ,  0.26849  , -0.30175  , -0.055796 ,
        0.12116  ,  0.010542 , -0.18065  ,  0.23281  , -0.26367  ,
        0.11032  ,  0.06216  ,  0.015019 , -0.10687  ,  0.098486 ,
        0.048457 ,  0.33355  , -0.16177  , -0.28503  , -0.28655  ,
       -0.11245  ,  0.12417  , -0.24975  , -0.2008   ,  0.26034  ,
        0.25208  , -0.17841  ,  0.15395  , -0.19799  , -0.22644  ,
       -0.074088 ,  0.50289  ,  0.32105  , -0.034766 ,  0.16543  ,
        0.057095 , -0.20973  ,  0.098376 ,  0.035058 , -0.023057 ,
       -0.11736  , -0.51327  , -0.020999 ,  0.39962  ,  0.30533  ,
       -0.38839  ,  0.0026097,  0.29022  ,  0.017045 ,  0.063961 ,
        0.10789  ,  0.29013  ,  0.061732 ,  0.068231 , -0.0140